In [67]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from tqdm import tqdm
tf.reset_default_graph()

In [68]:
def getdata():
    import pandas as pd
    df = pd.read_pickle('/home/patrick/Documents/Alt Coin Proj/data/newfeat.pkl')
    df = df['LTC']
    length = len(df)
    data = df.drop('up',axis=1)
    target = df['up']
    data = data.as_matrix()
    target = np.asarray(target, dtype=np.int8)
    
    
    N,M = data.shape
    
    all_X = np.ones((N,M+1))
    all_X[:,1:] = data
    
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels,dtype=np.int8)[target]
    #all_Y = np.array(target,dtype=np.int8)
    #all_Y = all_Y.reshape(len(all_Y),)
    
    trainx = all_X[2000:7000]
    testx = all_X[7001:]
    
    trainy = all_Y[2000:7000]
    testy = all_Y[7001:]
    
    
    return trainx,testx,trainy,testy,length

In [69]:
#Create a new log file
now=datetime.utcnow().strftime('%Y$m$d%H%M%S')
root_logdir = 'tf_logs1'
logdir = '{}/run-{}'.format(root_logdir,now)
    

In [70]:
#parameters

learning_rate = 0.005
training_epochs = 2000
batch_size = 100
display_step=100

#network pramenters

n_hidden_1 = 40
n_hidden_2 = 100
n_hidden_3 = 120
n_hidden_4 = 100
n_hidden_5 = 10
n_inputs = 25
n_classes = 2

In [71]:
trainx,testx,trainy,testy,length = getdata()
print (trainy.shape, length)


((5000, 2), 7763)


In [72]:
x = tf.placeholder("float", [None, n_inputs], name="X")
y = tf.placeholder("int64",[None,n_classes], name = 'Y') 


In [73]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
        W = tf.Variable(init, name='Weights')
        b = tf.Variable(tf.zeros([n_neurons]), name='Biases')
        z = tf.matmul(X,W)+b
        if activation == 'relu':
            return tf.nn.relu(z)
        elif activation == 'sig':
            return tf.nn.sigmoid(z)
        else:
            return z

In [74]:
with tf.name_scope('DNN'):
    hidden1 = neuron_layer(x,n_hidden_1, 'Hidden1', activation='relu')
    hidden2 = neuron_layer(hidden1, n_hidden_2, 'Hidden2', activation='relu')
    hidden3 = neuron_layer(hidden2, n_hidden_3, 'Hidden3', activation='relu')
    hidden4 = neuron_layer(hidden2, n_hidden_3, 'Hidden4', activation='relu')
    hidden5 = neuron_layer(hidden2, n_hidden_3, 'Hidden5', activation='sig')
    logits = neuron_layer(hidden3, n_classes,'outputs', )

In [75]:
with tf.name_scope('Loss'):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits = logits)
    loss = tf.reduce_mean(xentropy, name='Loss')

In [76]:
with tf.name_scope('Train'):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [77]:
with tf.name_scope('Eval'):
    correct = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [78]:
init = tf.global_variables_initializer()
#saver = tf.train.Saver()

In [79]:
with tf.name_scope('Batchdata'):
    x_bat, y_bat = tf.train.batch([trainx,trainy],enqueue_many=True,
                    batch_size=batch_size, capacity=8000)

loss_summary = tf.summary.scalar('Loss', loss)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
# Training of the Model
with tf.Session() as sess:
    init.run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    for epoch in range(training_epochs):
        for iteration in range(length/batch_size):
            x_batch, y_batch = sess.run([x_bat,y_bat])
            if iteration %10 ==0:
                summary_str= loss_summary.eval(feed_dict={x: x_batch,y:y_batch})
                step = epoch * int(length/batch_size)+iteration
                file_writer.add_summary(summary_str,step)
            #y_batch = y_batch.reshape([2,])
            #print y_batch
            sess.run(training_op, feed_dict = {x: x_batch,y:y_batch})
            
        acc_train = accuracy.eval(feed_dict = {x: x_batch,y:y_batch})
        acc_test = accuracy.eval(feed_dict = {x: testx,y:testy})
        
        if epoch%8 == 0:
            print(epoch, 'Train Accuracy: ', acc_train, 'Test Accuracy: ',acc_test)
    
    #save_path=saver.save(sess,'my_model_.ckpt')
    file_writer.close()
        
        

(0, 'Train Accuracy: ', 0.49000001, 'Test Accuracy: ', 0.53674543)
(8, 'Train Accuracy: ', 0.5, 'Test Accuracy: ', 0.53674543)
(16, 'Train Accuracy: ', 0.30000001, 'Test Accuracy: ', 0.53674543)
(24, 'Train Accuracy: ', 0.55000001, 'Test Accuracy: ', 0.53674543)
(32, 'Train Accuracy: ', 0.54000002, 'Test Accuracy: ', 0.53674543)
(40, 'Train Accuracy: ', 0.57999998, 'Test Accuracy: ', 0.53674543)
(48, 'Train Accuracy: ', 0.46000001, 'Test Accuracy: ', 0.53674543)
(56, 'Train Accuracy: ', 0.57999998, 'Test Accuracy: ', 0.53674543)
(64, 'Train Accuracy: ', 0.54000002, 'Test Accuracy: ', 0.53674543)
(72, 'Train Accuracy: ', 0.58999997, 'Test Accuracy: ', 0.53674543)
(80, 'Train Accuracy: ', 0.50999999, 'Test Accuracy: ', 0.53674543)
(88, 'Train Accuracy: ', 0.49000001, 'Test Accuracy: ', 0.53674543)
(96, 'Train Accuracy: ', 0.66000003, 'Test Accuracy: ', 0.53674543)
(104, 'Train Accuracy: ', 0.61000001, 'Test Accuracy: ', 0.53674543)
(112, 'Train Accuracy: ', 0.49000001, 'Test Accuracy: ',

(976, 'Train Accuracy: ', 0.55000001, 'Test Accuracy: ', 0.50918633)
(984, 'Train Accuracy: ', 0.54000002, 'Test Accuracy: ', 0.5)
(992, 'Train Accuracy: ', 0.50999999, 'Test Accuracy: ', 0.51181102)
(1000, 'Train Accuracy: ', 0.41999999, 'Test Accuracy: ', 0.50393701)
(1008, 'Train Accuracy: ', 0.44999999, 'Test Accuracy: ', 0.50524932)
(1016, 'Train Accuracy: ', 0.63999999, 'Test Accuracy: ', 0.50918633)
(1024, 'Train Accuracy: ', 0.5, 'Test Accuracy: ', 0.50918633)
(1032, 'Train Accuracy: ', 0.41999999, 'Test Accuracy: ', 0.50393701)
(1040, 'Train Accuracy: ', 0.56, 'Test Accuracy: ', 0.50918633)
(1048, 'Train Accuracy: ', 0.43000001, 'Test Accuracy: ', 0.50918633)
(1056, 'Train Accuracy: ', 0.41, 'Test Accuracy: ', 0.5065617)
(1064, 'Train Accuracy: ', 0.57999998, 'Test Accuracy: ', 0.51181102)
(1072, 'Train Accuracy: ', 0.52999997, 'Test Accuracy: ', 0.51574802)
(1080, 'Train Accuracy: ', 0.55000001, 'Test Accuracy: ', 0.50262469)
(1088, 'Train Accuracy: ', 0.55000001, 'Test Accur

(1952, 'Train Accuracy: ', 0.47, 'Test Accuracy: ', 0.48818898)
(1960, 'Train Accuracy: ', 0.47999999, 'Test Accuracy: ', 0.5)
(1968, 'Train Accuracy: ', 0.49000001, 'Test Accuracy: ', 0.49475065)
(1976, 'Train Accuracy: ', 0.56, 'Test Accuracy: ', 0.49606299)
(1984, 'Train Accuracy: ', 0.52999997, 'Test Accuracy: ', 0.49343833)
(1992, 'Train Accuracy: ', 0.47, 'Test Accuracy: ', 0.50393701)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: Batchdata/batch/fifo_queue_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_DOUBLE, DT_INT8], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](Batchdata/batch/fifo_queue, Batchdata/batch/Const, Batchdata/batch/Const_1)]]


In [621]:
def multilayer_preceptron(x,weighrs,biases):
    layer_1 = tf.add(tf.matmul(x,weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)

    
    out_layer = tf.matmul(layer_3, weights['out'])+ biases['out']
    return out_layer

In [622]:
weights = {
    'h1' : tf.Variable(tf.random_normal([n_inputs,n_hidden_1])),
    'h2' : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'h3' : tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    'out' : tf.Variable(tf.random_normal([n_hidden_3,n_classes]))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'b3' : tf.Variable(tf.random_normal([n_hidden_3])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [623]:
pred = multilayer_preceptron(x,weights, biases)

In [624]:
#loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(cost)

In [625]:
init = tf.global_variables_initializer()

In [626]:

imageBatch, labelBatch = tf.train.batch([trainx,trainy],enqueue_many=True,
                                        batch_size=batch_size, capacity=8000)




with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    #training is done HERE
    for epoch in tqdm(range(training_epochs)):
        batch_xs, batch_ys = sess.run([imageBatch,labelBatch])
        avg_cost = 0.
        total_batches = int(length/batch_size)
        #loop throough batches
        for i in range(total_batches):
            _, c = sess.run([optimizer,cost],feed_dict={x: batch_xs,
                                                      y: batch_ys})
            #compute avg loss
            avg_cost += c/total_batches
            
        #display the logs
        if epoch% display_step == 0:
            print('Epoch:', '%04d'%(epoch+1), 'cost=', '{:.9f}'.format(avg_cost))
    print('Optimization Complete!')
    
    #Test Model 
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    #calculate Accurarcy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: testx, y: testy}))


  0%|          | 4/2000 [00:00<02:32, 13.12it/s]

('Epoch:', '0001', 'cost=', 'nan')



  5%|▌         | 102/2000 [00:07<02:57, 10.68it/s]

('Epoch:', '0101', 'cost=', 'nan')



  6%|▌         | 112/2000 [00:08<02:52, 10.97it/s]

  7%|▋         | 136/2000 [00:10<02:08, 14.47it/s]

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: batch/fifo_queue_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_DOUBLE, DT_INT8], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](batch/fifo_queue, batch/Const, batch/Const_1)]]


KeyboardInterrupt: 

## 69% accuracy for DOGE

#parameters

learning_rate = 0.001
training_epochs = 10000
batch_size = 42
display_step=100

#network pramenters

n_hidden_1 = 250
n_hidden_2 = 500
n_hidden_3 = 250
n_inputs = 25
n_classes = 2